## Transfers Capstone - Data Wrangling

### Data Collection: 2016 - 2020 Transfers Data
Data sourced from: https://github.com/ewenme/transfers

### I. Setting up DataFrame

In [1]:
# Load packages and combine various seasons dataframes for PL and Championship into one DataFrame "df"
import pandas as pd
import glob
import os

os.chdir("data/original_data_sources/season_data")
extension = 'csv'
files = [i for i in glob.glob('*.{}'.format(extension))]

combined_data = pd.concat([pd.read_csv(f) for f in files])

combined_data.to_csv("combined_seasons_data.csv", index=False)

In [2]:
df = pd.read_csv('combined_seasons_data.csv')

### II. Examining the Data

In [3]:
# Quick look at a small segment of the dataframe
df.head()

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
0,Arsenal FC,Thomas,27,Defensive Midfield,Atlético Madrid,£45.00m,in,Summer,45.0,Premier League,2020,2020/2021
1,Arsenal FC,Gabriel,22,Centre-Back,LOSC Lille,£23.40m,in,Summer,23.4,Premier League,2020,2020/2021
2,Arsenal FC,Pablo Marí,26,Centre-Back,Flamengo,£7.20m,in,Summer,7.2,Premier League,2020,2020/2021
3,Arsenal FC,Rúnar Alex Rúnarsson,25,Goalkeeper,Dijon,£1.80m,in,Summer,1.8,Premier League,2020,2020/2021
4,Arsenal FC,Cédric Soares,28,Right-Back,Southampton,Free transfer,in,Summer,0.0,Premier League,2020,2020/2021


In [4]:
df.columns
# I'll keep all of these columns intact, as I think they'll be useful later

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season'],
      dtype='object')

In [5]:
df.dtypes
# 'Fee' is a feature with object types, 'fee_cleaned' contains float types.
# I will use 'fee_cleaned' for the best look at the amounts per transfer

club_name              object
player_name            object
age                     int64
position               object
club_involved_name     object
fee                    object
transfer_movement      object
transfer_period        object
fee_cleaned           float64
league_name            object
year                    int64
season                 object
dtype: object

In [6]:
df.info()
# 'fee_cleaned' has some null values in it I'll deal with later.
# 'transfer_period' looks like it might only be available for one of the two leagues, or for fewer seasons
# Not sure yet how to deal with 'transfer_period', might fill blanks with "unknown" so that I can still use column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17194 entries, 0 to 17193
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   club_name           17194 non-null  object 
 1   player_name         17194 non-null  object 
 2   age                 17194 non-null  int64  
 3   position            17194 non-null  object 
 4   club_involved_name  17194 non-null  object 
 5   fee                 17194 non-null  object 
 6   transfer_movement   17194 non-null  object 
 7   transfer_period     2400 non-null   object 
 8   fee_cleaned         15768 non-null  float64
 9   league_name         17194 non-null  object 
 10  year                17194 non-null  int64  
 11  season              17194 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 1.6+ MB


In [7]:
df.shape

(17194, 12)

In [8]:
df.nunique()

club_name               51
player_name           2698
age                     27
position                16
club_involved_name     803
fee                    718
transfer_movement        2
transfer_period          2
fee_cleaned            367
league_name              2
year                     5
season                   5
dtype: int64

In [9]:
# Looking at number of records per club
df.club_name.value_counts()

Nottingham Forest          588
Chelsea FC                 566
Wolverhampton Wanderers    564
Manchester City            492
Birmingham City            490
Reading FC                 488
Watford FC                 484
Brighton & Hove Albion     482
Norwich City               468
Leeds United               466
Bristol City               462
Aston Villa                424
Fulham FC                  422
Cardiff City               410
Barnsley FC                404
Queens Park Rangers        404
Huddersfield Town          398
Everton FC                 396
Swansea City               396
Newcastle United           392
Middlesbrough FC           368
Derby County               366
Wigan Athletic             362
Sheffield United           344
Preston North End          334
Stoke City                 334
Hull City                  332
AFC Bournemouth            322
West Ham United            320
Sheffield Wednesday        306
Crystal Palace             296
Brentford FC               294
Southamp

In [10]:
df.agg([min, max]).T
# A better look at the difference between 'fee' and 'fee_cleaned'
# 'fee_cleaned' is a float type, with 1.0 = £1mil.

,min,max
club_name,AFC Bournemouth,Wycombe Wanderers
player_name,Aapo Halme,Örjan Nyland
age,16,43
position,Attacking Midfield,Second Striker
club_involved_name,1. FC Köln,Östersund
fee,-,£990k
transfer_movement,in,out
fee_cleaned,0,130.5
league_name,Championship,Premier League
year,2016,2020


In [11]:
# From .info, I'll need to deal with 'fee_cleaned' and 'transfer_period'
df.fee_cleaned.fillna(0, inplace=True)
df.transfer_period.fillna('Unknown', inplace=True)

In [12]:
# Checking for duplicate rows
duplicateRowsDF = df[df.duplicated()]
duplicateRowsDF

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
3801,Swansea City,Marvin Emnes,28,Centre-Forward,Blackburn,Loan,out,Unknown,0.0,Premier League,2016,2016/2017
5395,Derby County,Max Lowe,21,Left-Back,Aberdeen FC,Loan,out,Unknown,0.0,Championship,2018,2018/2019
5908,Nottingham Forest,Zach Clough,23,Second Striker,Rochdale,Loan,out,Unknown,0.0,Championship,2018,2018/2019
6719,Preston North End,Eoin Doyle,29,Centre-Forward,Oldham Athletic,Loan,out,Unknown,0.0,Championship,2017,2017/2018
7035,Bristol City,Max O'Leary,21,Goalkeeper,Solihull Moors,Loan,out,Unknown,0.0,Championship,2017,2017/2018
...,...,...,...,...,...,...,...,...,...,...,...,...
17189,Barnsley FC,Samy Morsy,25,Central Midfield,Wigan,"End of loanJan 3, 2017",out,Unknown,0.0,Championship,2016,2016/2017
17190,Barnsley FC,Ryan Kent,20,Left Winger,Liverpool U23,"End of loanMay 31, 2017",out,Unknown,0.0,Championship,2016,2016/2017
17191,Barnsley FC,Matty James,25,Central Midfield,Leicester,"End of loanMay 31, 2017",out,Unknown,0.0,Championship,2016,2016/2017
17192,Barnsley FC,Gethin Jones,21,Right-Back,Everton U23,"End of loanMay 31, 2017",out,Unknown,0.0,Championship,2016,2016/2017


### III. Loading SPI Data to DataFrame

In [13]:
os.chdir('/home/tdraths/sb_assignments/Transfers_Capstone/data/original_data_sources')
spi_df = pd.read_csv('spi_matches.csv')
spi_df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
spi_df.columns
# I am not concerned with the match-specific columns toward the last half of this list
# I will likely only keep columns from 'season' through 'spi2'

Index(['season', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object')

In [15]:
spi_df.dtypes

season           int64
date            object
league_id        int64
league          object
team1           object
team2           object
spi1           float64
spi2           float64
prob1          float64
prob2          float64
probtie        float64
proj_score1    float64
proj_score2    float64
importance1    float64
importance2    float64
score1         float64
score2         float64
xg1            float64
xg2            float64
nsxg1          float64
nsxg2          float64
adj_score1     float64
adj_score2     float64
dtype: object

In [16]:
spi_df.league.value_counts()

English League Championship                 2223
French Ligue 1                              1900
Spanish Primera Division                    1900
Italy Serie A                               1900
Barclays Premier League                     1900
Spanish Segunda Division                    1865
Italy Serie B                               1594
English League Two                          1554
German Bundesliga                           1530
French Ligue 2                              1520
Brasileiro Série A                          1520
English League One                          1514
United Soccer League                        1487
Major League Soccer                         1459
Turkish Turkcell Super Lig                  1338
Portuguese Liga                             1224
German 2. Bundesliga                        1224
Dutch Eredivisie                            1224
Argentina Primera Division                   979
Russian Premier Liga                         960
Norwegian Tippeligae

In [17]:
# I'm only looking at the Premier League and English League Championship data for this project
ecl = spi_df[spi_df.league.eq('English League Championship')]
pl = spi_df[spi_df.league.eq('Barclays Premier League')]

english = pd.concat([ecl, pl])
display(english.head())
english.tail()


,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
2992,2017,2017-08-04,2412,English League Championship,Sunderland,Derby County,50.39,40.83,0.5266,0.2184,...,NaN,NaN,1.0,1.0,2.24,1.23,1.92,1.38,1.05,1.05
2994,2017,2017-08-04,2412,English League Championship,Nottingham Forest,Millwall,35.55,28.23,0.5149,0.2186,...,NaN,NaN,1.0,0.0,0.45,3.49,1.26,2.73,1.05,0.00
3004,2017,2017-08-05,2412,English League Championship,Sheffield United,Brentford,27.72,39.70,0.3031,0.4486,...,NaN,NaN,1.0,0.0,0.72,1.84,0.97,1.43,1.05,0.00
3005,2017,2017-08-05,2412,English League Championship,Queens Park Rangers,Reading,36.33,34.90,0.4420,0.2823,...,NaN,NaN,2.0,0.0,2.15,0.29,1.27,0.51,2.10,0.00
3006,2017,2017-08-05,2412,English League Championship,Fulham,Norwich City,43.00,42.60,0.4434,0.3142,...,NaN,NaN,1.0,1.0,1.19,1.71,2.35,1.88,1.05,1.05


,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
42041,2020,2021-05-23,2411,Barclays Premier League,Manchester City,Everton,94.16,75.80,0.7825,0.0736,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42042,2020,2021-05-23,2411,Barclays Premier League,Liverpool,Crystal Palace,91.38,69.71,0.7854,0.0661,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42043,2020,2021-05-23,2411,Barclays Premier League,Wolverhampton,Manchester United,78.46,85.38,0.3142,0.4122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42044,2020,2021-05-23,2411,Barclays Premier League,Arsenal,Brighton and Hove Albion,78.83,70.26,0.5391,0.2142,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42045,2020,2021-05-23,2411,Barclays Premier League,West Ham United,Southampton,69.01,70.94,0.4094,0.3385,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
display(english.league.value_counts())
english.season.value_counts()

English League Championship    2223
Barclays Premier League        1900
Name: league, dtype: int64

2019    937
2018    937
2017    937
2020    932
2016    380
Name: season, dtype: int64

In [19]:
english.columns

Index(['season', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object')

In [20]:
english_nulls = pd.DataFrame(english.isnull().sum().sort_values(ascending=False) / len(english), columns=['percent'])
eng_percent_null = english_nulls['percent'] > 0
english_nulls[eng_percent_null]

# High percentage of null values, but only in the columns that I am unlikely to need.

,percent
importance2,0.289595
importance1,0.289595
adj_score2,0.216832
nsxg2,0.216832
nsxg1,0.216832
xg2,0.216832
xg1,0.216832
adj_score1,0.216832
score2,0.215862
score1,0.215862


In [21]:
# Dropping the unnecessary columns. I'm only focused on the actual SPI ranking for clubs
# Match-specific data is not important for this analysis.
columns = ['importance2', 'importance1', 'adj_score2', 'nsxg2', 'nsxg1', 'xg2', 'xg1', 'adj_score1', 'score2', 'score1', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2']
english.drop(columns, inplace=True, axis=1)

english.head()

,season,date,league_id,league,team1,team2,spi1,spi2
2992,2017,2017-08-04,2412,English League Championship,Sunderland,Derby County,50.39,40.83
2994,2017,2017-08-04,2412,English League Championship,Nottingham Forest,Millwall,35.55,28.23
3004,2017,2017-08-05,2412,English League Championship,Sheffield United,Brentford,27.72,39.70
3005,2017,2017-08-05,2412,English League Championship,Queens Park Rangers,Reading,36.33,34.90
3006,2017,2017-08-05,2412,English League Championship,Fulham,Norwich City,43.00,42.60


##### I now have two datafames, one that houses cleaned transfers data for seasons 2016-2020 for the Championship and the Premier League, and another that houses cleaned SPI data for all English League Championship and Premier League clubs

In [22]:
df.to_csv("/home/tdraths/sb_assignments/Transfers_Capstone/data/data_cleaning_outputs/all_transfers.csv")
english.to_csv("/home/tdraths/sb_assignments/Transfers_Capstone/data/data_cleaning_outputs/english_output.csv")

### IV. Organizing the dataframes and some initial analysis
* Organize df to show spending by club by year
* Organize english to show spi score by club
* Combine the new organized dataframes into one workable dataframe

In [23]:
# Organizing df so that I can see what clubs spent in total across each season
df_agg_clubs = df.groupby(['year', 'club_name'], as_index=False).sum().loc[:, ['year', 'club_name', 'fee_cleaned']]
df_agg_clubs.groupby(['club_name']).count()

# I have a decision to make - should I continue to include the clubs that not in either league for at least one of the seasons we are investigating?

,year,fee_cleaned
club_name,,
AFC Bournemouth,5,5
Arsenal FC,5,5
Aston Villa,5,5
Barnsley FC,4,4
Birmingham City,5,5
Blackburn Rovers,4,4
Bolton Wanderers,2,2
Brentford FC,5,5
Brighton & Hove Albion,5,5


In [24]:
# Creating a dataframe showing the SPI score for the home team during each season
spi_home = english.groupby(['season', 'team1'], as_index=False).mean().loc[:, ['season', 'team1', 'spi1']]
spi_home.columns = ['year', 'club_name', 'spi_home']
spi_home

,year,club_name,spi_home
0,2016,AFC Bournemouth,60.945789
1,2016,Arsenal,82.174211
2,2016,Burnley,57.487368
3,2016,Chelsea,84.546842
4,2016,Crystal Palace,58.646842
...,...,...,...
191,2020,Watford,64.058261
192,2020,West Bromwich Albion,58.420526
193,2020,West Ham United,69.082105
194,2020,Wolverhampton,78.464211


In [25]:
# Creating a dataframe showing the SPI score for the away team during each season
spi_away = english.groupby(['season', 'team2'], as_index=False).mean().loc[:, ['season', 'team2', 'spi2']]
spi_away.columns = ['year', 'club_name', 'spi_away']
spi_away

,year,club_name,spi_away
0,2016,AFC Bournemouth,60.804211
1,2016,Arsenal,81.785789
2,2016,Burnley,57.947895
3,2016,Chelsea,84.528421
4,2016,Crystal Palace,58.364737
...,...,...,...
191,2020,Watford,64.019130
192,2020,West Bromwich Albion,58.345263
193,2020,West Ham United,68.997368
194,2020,Wolverhampton,78.435789


In [34]:
# Merging home and away dataframes into one that shows SPI scores per club per season
spi_scores = spi_home.merge(spi_away, how='inner', on=['year', 'club_name'])
spi_scores.head(40)

,year,club_name,spi_home,spi_away
0,2016,AFC Bournemouth,60.945789,60.804211
1,2016,Arsenal,82.174211,81.785789
2,2016,Burnley,57.487368,57.947895
3,2016,Chelsea,84.546842,84.528421
4,2016,Crystal Palace,58.646842,58.364737
5,2016,Everton,69.925789,70.190526
6,2016,Hull City,47.560000,47.643684
7,2016,Leicester City,62.715789,62.918947
8,2016,Liverpool,81.343684,80.923158
9,2016,Manchester City,86.021053,86.018947


In [33]:
# Merging df_agg_clubs and spi_scores to get one workable dataframe
avg_spend_spi = df_agg_clubs.merge(spi_scores, how='outer', on=['club_name', 'year'])
avg_spend_spi.head(50)

,year,club_name,fee_cleaned,spi_home,spi_away
0,2016,AFC Bournemouth,119.134,60.945789,60.804211
1,2016,Arsenal FC,222.112,NaN,NaN
2,2016,Aston Villa,236.710,NaN,NaN
3,2016,Barnsley FC,28.828,NaN,NaN
4,2016,Birmingham City,18.524,NaN,NaN
5,2016,Blackburn Rovers,23.860,NaN,NaN
6,2016,Brentford FC,35.190,NaN,NaN
7,2016,Brighton & Hove Albion,15.760,NaN,NaN
8,2016,Bristol City,48.106,NaN,NaN
9,2016,Burnley FC,84.240,NaN,NaN


## TO BE FIXED
* Club name issues - some of the clubs are listed as "FC" in one data set, and not in another

### STILL TO INVESTIGATE
* Total spend per year with changes in spi by year
* Total spend across years with total chain in spi
* Club results, e.g. promotion
* Look at clubs that were promoted into and relegated from PL as a subset